<a href="https://colab.research.google.com/github/mrkatey/code-gift-box-wrapper/blob/main/namemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install h3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00


In [13]:
import h3

# Input data
full_name = "Jeff Matkins"
birth_latitude = 34.05
birth_longitude = -118.25

# Convert latitude and longitude to H3 hexagon
h3_resolution = 12
h3_index = h3.geo_to_h3(birth_latitude, birth_longitude, h3_resolution)

# Print the H3 index
print("Full Name:", full_name)
print("H3 Index:", h3_index)


Full Name: Jeff Matkins
H3 Index: 8c29a1d752149ff


In [16]:
import csv
from pathlib import Path

def convert_coordinates_to_h3(csv_path, resolution):
    # Open the input CSV file
    with open(csv_path, 'r') as file:
        # Read the CSV data
        fieldnames = 'phoneme,event,year,latitude,longitude'.split(",")
        reader = csv.DictReader(file, fieldnames=fieldnames)
        header = next(reader)  # Read the header row
        rows = list(reader)    # Read the remaining rows
    
    # Convert latitude and longitude to H3 hexagons
    for row in rows:
        # Extract latitude and longitude from the row
        full_name = row['phoneme']
        latitude = float(row['latitude'])
        longitude = float(row['longitude'])
        
        # Convert latitude and longitude to H3 hexagon
        h3_index = h3.geo_to_h3(latitude, longitude, resolution)
        
        # Replace latitude and longitude with H3 hexagon in the row
        row[1] = h3_index
        del row[2]  # Remove the longitude column
    
    # Create a new CSV file for output
    output_path = 'output.csv'
    with open(output_path, 'w', newline='') as file:
        # Write the header row
        writer = csv.writer(file)
        writer.writerow(header)
        
        # Write the modified rows
        writer.writerows(rows)
    
    print(f"Conversion complete. Output CSV file saved at: {output_path}")

# Example usage
csv_path = Path(r"C:\Users\YW195YV\OneDrive - EY\sandbox\AutoML\geo_sound_index.csv")
resolution = 12           # Specify the desired H3 resolution

convert_coordinates_to_h3(csv_path, resolution)


FileNotFoundError: ignored

In [17]:
Path(r'https://data.sciencespo.fr/dataset.xhtml?persistentId=doi:10.21410/7E4/RDAG3O#')

PosixPath('https:/data.sciencespo.fr/dataset.xhtml?persistentId=doi:10.21410/7E4/RDAG3O#')

In [24]:
!wget https://data.sciencespo.fr/api/access/datafile/4432?gbrecs=true

--2023-05-18 11:53:34--  https://data.sciencespo.fr/api/access/datafile/4432?gbrecs=true
Resolving data.sciencespo.fr (data.sciencespo.fr)... 193.54.67.54
Connecting to data.sciencespo.fr (data.sciencespo.fr)|193.54.67.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261545417 (249M) [application/x-gzip]
Saving to: ‘4432?gbrecs=true’

4432?gbrecs=true    100%[===================>] 249.43M  1.41MB/s    in 3m 8s   

2023-05-18 11:56:43 (1.32 MB/s) - ‘4432?gbrecs=true’ saved [261545417/261545417]



In [81]:
import csv
import gzip
import io
from pathlib import Path
import h3

def parse_string_as_csv(string):
  reader = csv.reader(io.StringIO(string))
  return next(reader)
      

def clean_name(string):
  return string.split("(")[0].replace("_"," ")

on_error = 0

with gzip.open(Path('/content/names.csv.gz'), 'rb') as f:
  with open('training_data.csv', 'w') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=['name','year','h3_index'])
    writer.writeheader()
    header = parse_string_as_csv(next(f).decode('utf-8'))
    print(header)
    for count, row in enumerate(f):
      try:
        value = row.decode('utf-8')
        person = parse_string_as_csv(value)
        p = dict(zip(header, person))
        # name, birth_year, birth_latitude, birth_longitude
        name = clean_name(p['name'])
        birth_year = p['birth']
        death_year = p['death']
        try:
          birth_coords = [float(p[x]) for x in ['bpla1', 'bplo1']]
        except Exception as ee:
            continue
        try:
          death_coords = [float(p[x]) for x in ['dpla1', 'dplo1']]
        except Exception as ee:
            continue
        if death_year and all([x is not None and x != '' for x in birth_coords]):
          #write death coords, death year
          h3_index = h3_index = h3.geo_to_h3(*death_coords, resolution)
          writer.writerow(dict(zip(['name','year','h3_index'], [name, year,h3_index])))
        if birth_year and all([x is not None and x != '' for x in death_coords]):
          #write birth coords, birth year
          h3_index = h3_index = h3.geo_to_h3(*birth_coords, resolution)
          writer.writerow(dict(zip(['name','year','h3_index'], [name, year,h3_index])))
      except Exception as ee:
        on_error += 1
        if on_error % 1000 == 0:
          print(on_error)

['wikidata_code', 'birth', 'death', 'updated_death_date', 'approx_birth', 'approx_death', 'birth_min', 'birth_max', 'death_min', 'death_max', 'gender', 'level1_main_occ', 'name', 'un_subregion', 'birth_estimation', 'death_estimation', 'bigperiod_birth_graph_b', 'bigperiod_death_graph_b', 'curid', 'level2_main_occ', 'freq_main_occ', 'freq_second_occ', 'level2_second_occ', 'level3_main_occ', 'bigperiod_birth', 'bigperiod_death', 'wiki_readers_2015_2018', 'non_missing_score', 'total_count_words_b', 'number_wiki_editions', 'total_noccur_links_b', 'sum_visib_ln_5criteria', 'ranking_visib_5criteria', 'all_geography_groups', 'string_citizenship_raw_d', 'citizenship_1_b', 'citizenship_2_b', 'list_areas_of_rattach', 'area1_of_rattachment', 'area2_of_rattachment', 'list_wikipedia_editions', 'un_region', 'group_wikipedia_editions', 'bplo1', 'dplo1', 'bpla1', 'dpla1', 'pantheon_1', 'level3_all_occ']


In [91]:
import csv
from collections import Counter
from h3 import h3

def find_top_h3_indexes(dataset_file, input_string):
    # Step 1: Find the (up to) 500 closest strings
    closest_strings = []

    with open(dataset_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            name = row['name']
            if name.lower().startswith(input_string.lower()):
                closest_strings.append(name)

            if len(closest_strings) >= 500:
                break

    # Step 2: Find the top 3 most common h3_indexes for the closest strings
    with open(dataset_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        dataset = list(csv_reader)

    h3_indexes = [row['h3_index'] for row in dataset if row['name'] in closest_strings]
    top_h3_indexes = Counter(h3_indexes).most_common(3)

    return top_h3_indexes


In [115]:
import folium
from folium.plugins import HeatMap

def plot_results_on_map(top_h3_indexes, name_string):
    # Create a map centered at a specific location
    map_center = [0, 0]  # Specify the latitude and longitude for the map center
    map_zoom = 2  # Adjust the zoom level as needed
    map_instance = folium.Map(location=map_center, zoom_start=map_zoom)

    # Create a list to store the coordinates and weights
    heat_data = []

    # Add each h3 index as a heatmap point with the corresponding weight
    for h3_index, weight in top_h3_indexes:
        # Get the centroid coordinates of the h3 index
        centroid = h3.h3_to_geo(h3_index)

        # Append the coordinates and weight to the heat data list
        heat_data.append(centroid + (weight,))

    # Create a heatmap layer and add it to the map
    heatmap_layer = HeatMap(heat_data)
    heatmap_layer.add_to(map_instance)

    # Add the name_string to the HTML file
    name_html = f'<h3>{name_string}</h3>'
    map_instance.get_root().html.add_child(folium.Element(name_html))

    # Display the map
    return map_instance

def name_heatmap(name='Wolynko'):
  pth = Path(r"/content/training_data.csv")

  results = find_top_h3_indexes(pth, name)

  map_plot = plot_results_on_map(results, name)
  map_plot.save(f'/content/maps/{name}.html')

In [117]:
names = ['Cichocki','Kraszewski','Makarewicz','Stought']
for name in names:
  x = name_heatmap(name=name)
  print(x)

None
None
None
None


In [96]:
result

[('8c1f1d48864c7ff', 17), ('8c1f15ad31861ff', 15), ('8c1f8d7a498cdff', 8)]

In [113]:
!zip -r /content/maps.zip /content/maps

from google.colab import files

# Download the zip file
files.download('/content/maps.zip')


  adding: content/maps/ (stored 0%)
  adding: content/maps/Makarewicz.html (deflated 66%)
  adding: content/maps/Kraszewski.html (deflated 66%)
  adding: content/maps/Stought.html (deflated 66%)
  adding: content/maps/Cichocki.html (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [114]:
import folium
from folium.plugins import HeatMap

def plot_results_on_map(top_h3_indexes, name_string):
    # Create a map centered at a specific location
    map_center = [0, 0]  # Specify the latitude and longitude for the map center
    map_zoom = 2  # Adjust the zoom level as needed
    map_instance = folium.Map(location=map_center, zoom_start=map_zoom)

    # Create a list to store the coordinates and weights
    heat_data = []

    # Add each h3 index as a heatmap point with the corresponding weight
    for h3_index, weight in top_h3_indexes:
        # Get the centroid coordinates of the h3 index
        centroid = h3.h3_to_geo(h3_index)

        # Append the coordinates and weight to the heat data list
        heat_data.append(centroid + (weight,))

    # Create a heatmap layer and add it to the map
    heatmap_layer = HeatMap(heat_data)
    heatmap_layer.add_to(map_instance)

    # Add the name_string to the HTML file
    name_html = f'<h3>{name_string}</h3>'
    map_instance.get_root().html.add_child(folium.Element(name_html))

    # Display the map
    return map_instance

# Example usage
name_strings = ['Cichocki','Kraszewski','Makarewicz','Stought']

for name_string, top_h3_indexes in zip(name_strings, ):
    map_plot = plot_results_on_map(top_h3_indexes, name_string)
    map_plot.save(f'{name_string}_heatmap.html')


In [ ]:
find_top_h3_indexes(dataset_file, )